In [ ]:
def install(pkg):
  # for p in pkg:
    !pip.main(['install', pkg])

In [ ]:
install('numpy')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import array_to_img, img_to_array, load_img
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model, load_model
import cv2

import glob

In [ ]:
try:
    from google.colab import drive
    print("module 'google.colab' is installed")
except ModuleNotFoundError:
    print("module 'mutagen' is not installed")
    # or
    !pip install google-colab # the install function from the question

In [ ]:
drive.mount('/content/gdrive')

# Set the path to the dataset directory
train_dir = './data/cats/train'
validation_dir = './data/cats/test'


In [ ]:

# Set the image size and batch size
img_size = (299, 299)
batch_size = 32
num_classes=6


## Import data and create generators

In [ ]:
# Create the train generator
# Create image data generator with data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical')

# Create the validator generator
# Create image data generator without data augmentation for the validation set
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical')

## Sample images

In [ ]:
x_val_sample, _ = next(train_generator)
plt.figure(figsize=(12,8))

for i, x in enumerate(x_val_sample[:38]):
    ax = plt.subplot(5, 7, i+1)
    plt.imshow(x)
    plt.axis('off')
plt.show()

## Setup base model and layers

In [ ]:

# Load the InceptionV3 model pre-trained on ImageNet without the top layer
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add a global average pooling layer and a dense layer with 256 units for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)

# Add a final dense layer with the number of classes and a softmax activation
predictions = Dense(num_classes, activation='softmax')(x)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Combine the base model and the top layers to create the final model
model = Model(inputs=base_model.input, outputs=predictions)


# Compile the model with categorical crossentropy loss and Adam optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## Fit model

In [ ]:

# Train the model for 10 epochs
num_epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch_size)
# Save the model
model.save('cat_breed_model.h5')


In [ ]:
model.summary()

In [ ]:
breed_names = {
    0: 'American Shorthair',
    1: 'Bengal',
    2: 'Maine Coon',
    3: 'Ragdoll',
    4: 'Scottish Fold',
    5: 'Sphynx'
}

## Plot the performance of the model

In [ ]:

# datapoints = model.history.history

# history = {
#   'train_acc': ['Training accuracy', datapoints['accuracy']],
#   'test_acc': ['Validation accuracy', datapoints['val_accuracy']],
#   'train_loss': ['Training loss', datapoints['loss']],
#   'test_loss': ['Validation loss', datapoints['val_loss']]
# }




In [ ]:
def plot_graph(history):
  title = 'Training/validation loss and accuracy'
  
  epochs = range(len(history['accuracy']))
  plt.title(title)
  plt.plot(epochs, history['accuracy'], label="Training accuracy")
  plt.plot(epochs, history['val_accuracy'], label="Validation accuracy")
  plt.plot(epochs, history['loss'], label="Training loss")
  plt.plot(epochs, history['val_loss'], label="Validation loss")
  plt.legend()
  plt.show()

In [ ]:
plot_graph(history.history)

## Evaluate the cat breed classification model

In [ ]:
model = load_model('cat_breed_classification_model.h5')

X_val_sample, _ = next(validation_generator)
y_pred = model.predict(X_val_sample)
plt.figure(figsize=(12,6))


nb_sample = 10
for i, (x, y) in enumerate(zip(X_val_sample[:nb_sample], y_pred[:nb_sample])):
    ax = plt.subplot(2, 5, i+1)
    predicted_class = np.argmax(y)
    breed_name = breed_names[predicted_class]
    title = 'Breed: '+breed_name + '\n Accuracy: ' + str(np.max(y))
    plt.imshow(x)
    plt.title(title)
    plt.axis('off')
plt.show()

## Algorithm to detect human face or cat breed

In [ ]:
# import trained inceptionV3 model
model = load_model('cat_breed_classification_model.h5')
# import pretrained human face cascade
face_cascade = cv2.CascadeClassifier("./haarcascade_frontalface_default.xml")

# preprocessing images
def tensor_3d_to_4d(img):
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def detect_human_face(img):
  global face_cascade
  isHuman = False
  print(type(img))
  
  if (img is None): return isHuman
  
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  face = face_cascade.detectMultiScale(gray, 1.3, 5)

  for (x,y,w,h) in face:
      img = cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
      isHuman = True

  return isHuman

def detect_cat_breed(img):
  global model
  y = model.predict(tensor_3d_to_4d(img))
  
  if (np.argmax(y) < 0.2): return False
  
  result = {}
  predicted_class = np.argmax(y)
  result['breed_name'] = breed_names[predicted_class]
  result['accuracy'] = np.max(y)
  
  return result
  
def eval_image(img):
  cat_breed = detect_cat_breed(img)
  
  if (detect_human_face(img)):
    return 'You\'re human\n'
  elif (cat_breed):
    return cat_breed['breed_name'] +' breed of cat\nI\'m '+ str(cat_breed['accuracy']) +' sure'
  else:
    return 'Not human / cat\n'
  
  

## Test algorithm with human faces

In [ ]:
humans = [cv2.imread(file) for file in glob.glob("data/human/*.jpg")]

plt.figure(figsize=(12,8))

for i, image in enumerate(list(humans[0:10])):
  image = cv2.resize(image, (224, 224))
  ax = plt.subplot(3, 5, i+1)
  title = eval_image(image)

  cv_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  plt.imshow(cv_rgb)
  plt.title(title)
  plt.axis("off")
plt.show()

## Test the Algorithm

In [ ]:
cats = [cv2.imread(file) for file in glob.glob("./data/different_cat_breeds/*.jpg")]

plt.figure(figsize=(15,10))

for i, image in enumerate(cats[:10]):
  if (image is None): continue
  
  ax = plt.subplot(3, 5, i+1)
  
  title = eval_image(image)
  print(title)
  cv_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  plt.imshow(image)
  plt.title(title)
  plt.axis("off")
plt.show()

## Comparison with other models (VGG16)

In [ ]:
# Load pre-trained VGG16 model without top layers
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze pre-trained layers
for layer in vgg.layers:
    layer.trainable = False

# Add custom classification layers
x = vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Compile model
vgg_model = Model(inputs=vgg.input, outputs=predictions)
vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history = vgg_model.fit(train_generator, epochs=10, validation_data=validation_generator)

In [ ]:
plot_graph(history.history)

In [ ]:
vgg_model.summary()